<a href="https://colab.research.google.com/github/ashkanb77/cats-vs-dogs/blob/main/CatVSDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

 99% 777M/788M [00:05<00:00, 140MB/s]
100% 788M/788M [00:05<00:00, 139MB/s]


In [3]:
def prepare_dataset(test_size):
  import zipfile
  import os
  import shutil
  from random import randint

  zipfile.ZipFile('microsoft-catsvsdogs-dataset.zip').extractall('cats_vs_dogs')

  os.makedirs('cats_vs_dogs/train')

  shutil.move('cats_vs_dogs/PetImages/Cat', 'cats_vs_dogs/train')
  shutil.move('cats_vs_dogs/PetImages/Dog', 'cats_vs_dogs/train')

  os.makedirs('cats_vs_dogs/test/Cat')
  os.makedirs('cats_vs_dogs/test/Dog')

  l = os.listdir('cats_vs_dogs/train/Cat')
  n = int(test_size * len(l))
  for i in range(n):
    r = l.pop(randint(0, len(l) - 1))
    shutil.move('cats_vs_dogs/train/Cat/' + r, 'cats_vs_dogs/test/Cat/' + r)
  

  l = os.listdir('cats_vs_dogs/train/Dog')
  n = int(test_size * len(l))
  for i in range(n):
    r = l.pop(randint(0, len(l) - 1))
    shutil.move('cats_vs_dogs/train/Dog/' + r, 'cats_vs_dogs/test/Dog/' + r)

prepare_dataset(0.1)

In [28]:
import os
from PIL import Image
folder_path = 'cats_vs_dogs/test'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])
    

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

In [25]:
os.remove('cats_vs_dogs/train/Cat/Thumbs.db')

In [29]:
data_generator = ImageDataGenerator(
    rescale=1/255, zoom_range=0.1, 
    horizontal_flip=True, vertical_flip=True,
    rotation_range=50, width_shift_range=0.1, 
    height_shift_range=0.1, shear_range=0.1, fill_mode='nearest'
    )

In [30]:
train = data_generator.flow_from_directory('cats_vs_dogs/train', class_mode='binary')

Found 22498 images belonging to 2 classes.


In [31]:
test = data_generator.flow_from_directory('cats_vs_dogs/test', class_mode='binary')

Found 2500 images belonging to 2 classes.


In [60]:
model = Sequential([
                    Conv2D(32, (9, 9), activation='relu', input_shape=(256, 256, 3)),
                    MaxPool2D(),
                    Conv2D(64, (5, 5), activation='relu'),
                    MaxPool2D(),
                    Conv2D(32, (3, 3), activation='relu'),
                    MaxPool2D(),
                    Flatten(),
                    Dense(256, activation='relu'),
                    Dense(512, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [61]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 248, 248, 32)      7808      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 124, 124, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 120, 120, 64)      51264     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 60, 60, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 58, 58, 32)        18464     
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 29, 29, 32)     

In [62]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [63]:
model.fit(train, steps_per_epoch=len(train), validation_data=test, validation_steps=len(test), epochs=5)

Epoch 1/5
704/704 [==============================] - ETA: 0s - loss: 0.7002 - accuracy: 0.5207

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

704/704 [==============================] - 404s 573ms/step - loss: 0.7002 - accuracy: 0.5207 - val_loss: 0.6931 - val_accuracy: 0.5020
Epoch 2/5
704/704 [==============================] - 398s 566ms/step - loss: 0.6764 - accuracy: 0.5701 - val_loss: 0.6493 - val_accuracy: 0.6224
Epoch 3/5
704/704 [==============================] - 391s 556ms/step - loss: 0.6473 - accuracy: 0.6263 - val_loss: 0.6232 - val_accuracy: 0.6464
Epoch 4/5
704/704 [==============================] - 393s 558ms/step - loss: 0.6323 - accuracy: 0.6456 - val_loss: 0.6296 - val_accuracy: 0.6504
Epoch 5/5
704/704 [==============================] - 390s 554ms/step - loss: 0.6218 - accuracy: 0.6603 - val_loss: 0.6281 - val_accuracy: 0.6536
